# Evaluation funtion

In [30]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertConfig
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()
            
def sentencepair2tensor(tokenizer, tokens_a, tokens_b, max_seq_length):
    
    _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
    
    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)

    assert len(tokens_b) > 0
    for token in tokens_b:
        tokens.append(token)
        segment_ids.append(1)
    tokens.append("[SEP]")
    segment_ids.append(1)
    masked_index = tokens.index("[MASK]")
    sep_index = tokens.index("[MASK]")
    

    
    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    
    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(segment_ids) == max_seq_length
    
    tokens_tensor = torch.tensor([input_ids])
    segments_tensors = torch.tensor([segment_ids])
    
    return tokens_tensor, segments_tensors, masked_index, sep_index

In [3]:
df = pd.read_csv('data/generation/BC_test.csv')

In [4]:
# Load pre-trained model tokenizer (vocabulary)
modelpath = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(modelpath)

In [7]:
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained(modelpath)
model.eval()


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertIntermediate(
       

In [19]:
def eval_naive(model,
               sentence1,
               sentence2,
               label,
               tokenizer,
               max_seq_length,
               top=10):

    sentence1 = tokenizer.tokenize(sentence1)
    sentence2 = tokenizer.tokenize(sentence2)
    target = tokenizer.convert_tokens_to_ids([label])[0]
    tokens_tensor, segments_tensors, masked_index, sep_index = sentencepair2tensor(tokenizer,
                                                                                   sentence1,
                                                                                   sentence2,
                                                                                   max_seq_length)
    predictions = model(tokens_tensor, segments_tensors)
    _, indices = torch.topk(predictions[0, masked_index], top)
    indices = list(indices.numpy())
    try:
        v = 1/ (indices.index(target) + 1)
    except ValueError:
        v = 0
    return v

##  Evaluating the pre-trained model

In [21]:
values = [] 

for a,b,l in tqdm(zip(df.sentence1.values,df.sentence2_masked.values,df.label.values)):
    v = eval_naive(model=model,
                   sentence1=a,
                   sentence2=b,
                   label=l,
                   tokenizer=tokenizer,
                   max_seq_length=128)
    values.append(v)
    

In [22]:
df["bert_base_uncased_pre_trained"] = values

In [27]:
np.mean(df["bert_base_uncased_pre_trained"].values)

0.16402103174603172

## Evaluating model withou pre-training

In [31]:
config = BertConfig(vocab_size_or_config_json_file=30522, hidden_size=768,
                    num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072)

model = BertForMaskedLM(config)
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertIntermediate(
       

In [32]:
values = [] 

for a,b,l in tqdm(zip(df.sentence1.values,df.sentence2_masked.values,df.label.values)):
    v = eval_naive(model=model,
                   sentence1=a,
                   sentence2=b,
                   label=l,
                   tokenizer=tokenizer,
                   max_seq_length=128)
    values.append(v)
    

1000it [01:58,  8.44it/s]


In [33]:
df["bert_base_uncased_no_pre_trained"] = values

In [34]:
np.mean(df["bert_base_uncased_no_pre_trained"].values)

0.0

In [35]:
df.to_csv("results/basic.csv")